# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [2]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.content)
else:
    print(req.status_code)

b'"Alive"'


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [3]:
# Set the countries_url variable (1 line)
countries_url = root_url + "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code)
print(countries)

403
{'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [4]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + "/cookie"
# Query the endpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url).cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=e6b7858f-8beb-43ed-ba49-367ee7aa7258 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [5]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()
# display the countries variable (1 line)
print(countries)

['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [6]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies)
# display the leaders variable (1 line)
print(leaders.content)

b'{"message":"Please specify a country"}'


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params={"country":"be"}).json()
# display the leaders variable (1 line)
print(leaders)

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines

# Initialize dict
leaders_per_country = {}
# Loop through 'countries' list
for country in countries:
    # Query the /leaders endpoint for each country + 
    # Save each json result in dict 'leaders_per_country'
    leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()

print(leaders_per_country)

In [ ]:
# or 1 line
leaders_per_country = {
    country: requests.get(leaders_url, cookies=cookies, params={"country": country}).json() for country in countries
}
    
print(leaders_per_country)

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [10]:
# < 15 lines
def get_leaders() -> dict:
    """
    Gets data of leaders of countries retrieved from "https://country-leaders.onrender.com"

    Args: None

    Returns:
        dict: Dictionary of countries (abbreviations) and their respective leaders' information

    """
    # Define URLs
    root_url = "https://country-leaders.onrender.com"
    countries_url = root_url + "/countries"
    cookie_url = root_url + "/cookie"
    leaders_url = root_url + "/leaders"
    # Get Cookies
    cookies = requests.get(cookie_url).cookies
    # Get Countries
    countries = requests.get(countries_url, cookies=cookies).json()
    # Loop over countries and save their leaders' data in a dictionary
    leaders_per_country = {
    country: requests.get(leaders_url, cookies=cookies, params={"country": country}).json() for country in countries
    }
    # Return the dictionary
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
# 3 lines

# Get wiki url of first leader of first country
wiki_url_us1 = leaders_per_country["us"][0]["wikipedia_url"]
# Query url
text = requests.get(wiki_url_us1)
# Display its text
print(text.text)

#### (Creating list of wiki urls.)

In [ ]:
# Get cookies
cookies = requests.get(cookie_url).cookies
# Get list of countries
countries = requests.get(countries_url, cookies=cookies).json()
# Iterate through dictionary and get list of wikipedia_urls
wiki_urls = []
for country in countries:
    # Iterate through dict values (one list per country) -> through dicts per leader
    for dict in leaders_per_country[country]:
        wiki_urls.append(dict["wikipedia_url"])

print(wiki_urls)

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
from bs4 import BeautifulSoup

# 3 lines
r = requests.get(wiki_url_us1)
soup = BeautifulSoup(r.content, "html")
text_content = soup.get_text()

print(soup.prettify())

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [15]:
# 2 lines
paragraphs = [tag for tag in soup.find_all("p")]
paragraphs[1]

<p><b>George Washington</b> (February 22, 1732 – <span class="nowrap">December 14</span>, 1799) was a <a class="mw-redirect" href="/wiki/Founding_Father_of_the_United_States" title="Founding Father of the United States">Founding Father of the United States</a>, military officer, and farmer who served as the first <a href="/wiki/President_of_the_United_States" title="President of the United States">president of the United States</a> from 1789 to 1797. Appointed by the <a href="/wiki/Second_Continental_Congress" title="Second Continental Congress">Second Continental Congress</a> as commander of the <a href="/wiki/Continental_Army" title="Continental Army">Continental Army</a> in 1775, Washington led <a href="/wiki/Patriot_(American_Revolution)" title="Patriot (American Revolution)">Patriot</a> forces to victory in the <a href="/wiki/American_Revolutionary_War" title="American Revolutionary War">American Revolutionary War</a> and then served as president of the <a href="/wiki/Constitution

##### Comparison: Wiki page where the first paragraph does not have index 1

In [16]:
wiki_url_fr = leaders_per_country["fr"][0]["wikipedia_url"]
r2 = requests.get(wiki_url_fr)
soup2 = BeautifulSoup(r2.content, "html")
paragraphs2 = [tag.text for tag in soup2.find_all("p")]
print(paragraphs2[3])

François Hollande [fʁɑ̃swa ɔlɑ̃d][n 3] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [17]:
# <10 lines

# The right paragraph in the list is the only one with <b> 
# Get index of paragraph -> next = 'nearest'
par_index = next(
    (i for i, tag in enumerate(paragraphs) if tag.find("b") is not None),
    None
)
# Get the text of the paragraph at this index
first_paragraph = paragraphs[par_index].text
first_paragraph

'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [18]:
# 10 lines

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)                                                       # keep this for the rest of the notebook
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.content, "html")
    paragraphs = [tag for tag in soup.find_all("p")]
    par_index = next(
        (i for i, tag in enumerate(paragraphs) if tag.find("b") is not None),
        None
    )
    return paragraphs[par_index].text

In [19]:
# Test: 3 lines
first_par_us_1 = get_first_paragraph("https://en.wikipedia.org/wiki/George_Washington")
print(first_par_us_1)

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".



In [20]:
# Test 2
first_par_fr_1 = get_first_paragraph(wiki_url_fr)
print(first_par_fr_1)

https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Hollande [fʁɑ̃swa ɔlɑ̃d][n 3] Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



#### (Get first paragraph for all leaders)

In [ ]:
first_paragraphs = []

for url in wiki_urls:
    first_paragraphs.append(get_first_paragraph(url))

print(first_paragraphs)

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re
def clean_paragraph(text):
    # Remove specific unwanted patterns:
    cleaned_text = re.sub(
        r'\[.*?\]'                              # Content within square brackets
        r'|/[^/]+/;?'                           # Content between slashes with optional trailing semicolon
        r'|(?:\b(?:Écouter|ⓘ|uitspraak)\b;?)'  # Remove "Écouter", "ⓘ" and "uitspraak" optionally followed by ";"
        r'|;\s*$',                              # Standalone semicolon at the end of the text or line
        '', 
        text
    )

    # Normalize spaces around punctuation and parentheses
    cleaned_text = re.sub(r'\(\s+', '(', cleaned_text)  # Remove space after opening parentheses
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Normalize whitespace
    
    # Remove any empty parentheses and space before commas at the end
    cleaned_text = re.sub(r'\(\s*\)', '', cleaned_text)  # Empty parentheses like "()" or "( )"
    cleaned_text = re.sub(r'\s+,', ',', cleaned_text)  # Remove spaces before commas
    
    return cleaned_text

print(clean_paragraph(first_par_fr_1))

François Hollande, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.


In [ ]:
for par in first_paragraphs:
    print(clean_paragraph(par))
    print()

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [24]:
# 10 lines
from typing import Optional

def get_first_paragraph(wikipedia_url: str) -> Optional[str]:
    """
    Fetches the first paragraph from a Wikipedia URL using a given session.

    Args:
        wikipedia_url (str): The Wikipedia URL of the respective state leader.
        session (requests.Session): A session object to make the request.

    Returns:
        Optional[str]: Cleaned first paragraph from the Wikipedia page, or None if not found.
    """
    
    # print(wikipedia_url)                      #Keep this for the rest of the notebook (TO DO)

    # Use session to make request
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.content, "html")
    # Get list of paragraphs
    paragraphs = [tag for tag in soup.find_all("p")]
    # Get index of first paragraph
    par_index = next(
        (i for i, tag in enumerate(paragraphs) if tag.find("b") is not None),
        None
    )
    # Clean paragraph and return, if found, otherwise return None
    return clean_paragraph(paragraphs[par_index].text) if par_index is not None else None

In [ ]:
# Test updated get_first_paragraph function
for url in wiki_urls:
    print(get_first_paragraph(url))
    print()

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [26]:
# < 20 lines
import re
from typing import Optional

def clean_paragraph(text: str) -> str:
    """
    Removes unwanted patterns from text.

    Args:
        text(str): Wikipedia paragraph with patterns to be removed, such as phonetic script, web symbols, remaining empty brackets or whitespace.

    Returns:
        cleaned_text(str): Cleaned wikipedia paragraph.
    """
    
    # Remove specific unwanted patterns:
    cleaned_text = re.sub(
        r'\[.*?\]'                # Content within square brackets
        r'|/[^/]+/;?'             # Content between slashes with optional trailing semicolon
        r'|(?:\b(?:Écouter|uitspraak)\b;?)'  # Remove "Écouter" and "uitspraak" optionally followed by ";"
        r'|;\s*$',                # Standalone semicolon at the end of the text or line
        '', 
        text
    )
    # Remove remaining ⓘ
    cleaned_text = re.sub(r'ⓘ', '', cleaned_text)

    # Normalize spaces around punctuation and parentheses
    cleaned_text = re.sub(r'\(\s+', '(', cleaned_text)  # Remove space after opening parentheses
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Normalize whitespace
    
    # Remove any empty parentheses and space before commas at the end
    cleaned_text = re.sub(r'\(\s*\)', '', cleaned_text)  # Empty parentheses like "()" or "( )"
    cleaned_text = re.sub(r'\s+,', ',', cleaned_text)  # Remove spaces before commas
    cleaned_text = re.sub(r'\(\s*;\s*', '(', cleaned_text)  # Specifically remove "; " in parentheses
    
    return cleaned_text


def get_first_paragraph(wikipedia_url: str) -> Optional[str]:
    """
    Fetches the first paragraph from a Wikipedia URL using a given session.

    Args:
        wikipedia_url (str): The Wikipedia URL of the respective state leader.
        session (requests.Session): A session object to make the request.

    Returns:
        Optional[str]: Cleaned first paragraph from the Wikipedia page, or None if not found.
    """
    
    # print(wikipedia_url)                      #Keep this for the rest of the notebook (TO DO)

    # Use session to make request
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.content, "html")
    # Get list of paragraphs
    paragraphs = [tag for tag in soup.find_all("p")]
    # Get index of first paragraph
    par_index = next(
        (i for i, tag in enumerate(paragraphs) if tag.find("b") is not None),
        None
    )
    # Clean paragraph and return, if found, otherwise return None
    return clean_paragraph(paragraphs[par_index].text) if par_index is not None else None

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [31]:
# < 20 lines
def get_leaders() -> dict:
    """
    Gets data of country leaders retrieved from "https://country-leaders.onrender.com",
    accesses wiki page, gets and cleans first paragraph.

    Args: None

    Returns:
        dict: Dictionary of countries and information on their respective leaders incl. first paragraph of wikipedia article.
    """
    # Define URLs
    root_url = "https://country-leaders.onrender.com"
    countries_url = root_url + "/countries"
    cookie_url = root_url + "/cookie"
    leaders_url = root_url + "/leaders"
    # Get Cookies
    cookies = requests.get(cookie_url).cookies
    # Get Countries
    countries = requests.get(countries_url, cookies=cookies).json()
    # Initialize dictionary
    leaders_per_country = {}
    # Outer loop: Iterate over countries
    for country in countries:
        # Get leaders' data and add to dict
        leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()
        # Inner loop: Iterate over leaders
        for leader_dict in leaders_per_country[country]:
            # Get wiki url
            wiki_url = leader_dict["wikipedia_url"]
            # Get and clean first paragraph
            first_paragraph = get_first_paragraph(wiki_url)
            # Add paragraph to leaders dictionary
            leader_dict["wiki_paragraph"] = first_paragraph
    
    return leaders_per_country

In [32]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country)

TypeError: string indices must be integers, not 'str'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [33]:
# < 25 lines
def get_leaders() -> dict:
    """
    Gets data of country leaders retrieved from "https://country-leaders.onrender.com",
    accesses wiki page, gets and cleans first paragraph.

    Args: None

    Returns:
        dict: Dictionary of countries and information on their respective leaders incl. first paragraph of wikipedia article.
    """
    # Define URLs
    root_url = "https://country-leaders.onrender.com"
    countries_url = root_url + "/countries"
    cookie_url = root_url + "/cookie"
    leaders_url = root_url + "/leaders"

    # Get Cookies
    cookies = requests.get(cookie_url).cookies
    
    # Get Countries
    countries = requests.get(countries_url, cookies=cookies).json()
    
    # Initialize dictionary
    leaders_per_country = {}
    
    # Outer loop: Iterate over countries
    for country in countries:

        # Check, if the cookies are expired (response code 403 instead of 200)
        response = requests.get(leaders_url, cookies=cookies, params={"country":country})

        if response.status_code != 200:
            # Refresh cookies
            cookies = requests.get(cookie_url).cookies
            # Renew request
            leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()
        else:
            # Use response data: Get leaders' data and add to dict
            leaders_per_country[country] = response.json()

        # Inner loop: Iterate over leaders
        for leader_dict in leaders_per_country[country]:
            # Get wiki url
            wiki_url = leader_dict["wikipedia_url"]
            # Use session to get the first paragraph from Wiki (cleaning incl.)
            first_paragraph = get_first_paragraph(wiki_url)
            # Add cleaned paragraph to leaders dictionary
            leader_dict["wiki_paragraph"] = first_paragraph
    
    return leaders_per_country


Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
print(get_leaders())

In [ ]:
# Test execution time

import time

start_time = time.time()
leaders_per_country = get_leaders()
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 49.15 seconds


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [45]:
# < 20 lines
from typing import Optional

def get_first_paragraph(wikipedia_url: str, session: requests.Session) -> Optional[str]:
    """
    Fetches the first paragraph from a Wikipedia URL using a given session.

    Args:
        wikipedia_url (str): The Wikipedia URL of the respective state leader.
        session (requests.Session): A session object to make the request.

    Returns:
        Optional[str]: Cleaned first paragraph from the Wikipedia page, or None if not found.
    """
    # print(wikipedia_url)                      #Keep this for the rest of the notebook (TO DO)

    # Use session to make request
    req = session.get(wikipedia_url)
    soup = BeautifulSoup(req.content, "html")
    # Get list of paragraphs
    paragraphs = [tag for tag in soup.find_all("p")]
    # Get index of first paragraph
    par_index = next(
        (i for i, tag in enumerate(paragraphs) if tag.find("b") is not None),
        None
    )
    # Clean paragraph and return, if found, otherwise return None
    return clean_paragraph(paragraphs[par_index].text) if par_index is not None else None

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [46]:
# <25 lines
def get_leaders() -> dict:
    """
    Gets data of country leaders retrieved from "https://country-leaders.onrender.com",
    accesses wiki page using session , gets and cleans first paragraph.

    Args: None

    Returns:
        dict: Dictionary of countries and information on their respective leaders incl. first paragraph of wikipedia article.
    """
    # Define URLs
    root_url = "https://country-leaders.onrender.com"
    countries_url = root_url + "/countries"
    cookie_url = root_url + "/cookie"
    leaders_url = root_url + "/leaders"

    # Create a session for Wikipedia requests
    session = requests.Session()

    # Get Cookies
    cookies = requests.get(cookie_url).cookies
    
    # Get Countries
    countries = requests.get(countries_url, cookies=cookies).json()
    
    # Initialize dictionary
    leaders_per_country = {}
    
    # Outer loop: Iterate over countries
    for country in countries:

        # Check, if the cookies are expired (response code 403 instead of 200)
        response = requests.get(leaders_url, cookies=cookies, params={"country":country})

        if response.status_code != 200:
            # Refresh cookies
            cookies = requests.get(cookie_url).cookies
            # Renew request
            leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()
        else:
            # Use response data: Get leaders' data and add to dict
            leaders_per_country[country] = response.json()

        # Inner loop: Iterate over leaders
        for leader_dict in leaders_per_country[country]:
            # Get wiki url
            wiki_url = leader_dict["wikipedia_url"]
            # Use session to get the first paragraph from Wiki (cleaning incl.)
            first_paragraph = get_first_paragraph(wiki_url, session)
            # Add cleaned paragraph to leaders dictionary
            leader_dict["wiki_paragraph"] = first_paragraph
    
    return leaders_per_country

Test your new functions.



In [ ]:
print(get_leaders())

In [47]:
# Test execution time

import time

start_time = time.time()
leaders_per_country = get_leaders()
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 43.01 seconds


#### (Test: Using concurrent.futures to enhance execution time)

In [43]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Optional

def get_first_paragraph(wikipedia_url: str, session: requests.Session) -> Optional[str]:
    req = session.get(wikipedia_url)
    soup = BeautifulSoup(req.content, "html.parser")
    paragraphs = [tag for tag in soup.find_all("p")]
    par_index = next((i for i, tag in enumerate(paragraphs) if tag.find("b") is not None), None)
    return clean_paragraph(paragraphs[par_index].text) if par_index is not None else None

def fetch_paragraph_with_session(leader_dict, session):
    wiki_url = leader_dict["wikipedia_url"]
    first_paragraph = get_first_paragraph(wiki_url, session)
    leader_dict["wiki_paragraph"] = first_paragraph
    return leader_dict

def get_leaders() -> dict:
    root_url = "https://country-leaders.onrender.com"
    countries_url = root_url + "/countries"
    cookie_url = root_url + "/cookie"
    leaders_url = root_url + "/leaders"
    session = requests.Session()
    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies).json()
    leaders_per_country = {}

    for country in countries:
        response = requests.get(leaders_url, cookies=cookies, params={"country": country})
        if response.status_code != 200:
            cookies = requests.get(cookie_url).cookies
            response = requests.get(leaders_url, cookies=cookies, params={"country": country})
        
        leaders_per_country[country] = response.json()

        # Use ThreadPoolExecutor to fetch paragraphs in parallel
        with ThreadPoolExecutor(max_workers=5) as executor:  # Experiment with max_workers (e.g., 5 or 10)
            futures = [executor.submit(fetch_paragraph_with_session, leader_dict, session) for leader_dict in leaders_per_country[country]]

            for future in as_completed(futures):
                try:
                    leader = future.result()
                except Exception as e:
                    print(f"Error fetching data: {e}")

    return leaders_per_country


In [ ]:
# Test execution time

import time

start_time = time.time()
leaders_per_country = get_leaders()
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.2f} seconds")

Execution time: 36.31 seconds


## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [48]:
# 3 lines
import json

with open("leaders.json", "w") as json_file:
    json.dump(leaders_per_country, json_file, indent=4)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same time.

In [49]:
# 3 lines

# Read the data back from leaders.json
with open("leaders.json", "r") as json_file:
    loaded_data = json.load(json_file)

# Check if the content is the same as the original dictionary
if loaded_data == leaders_per_country:
    print("Variable content matches. The file was saved and loaded successfully.")
else:
    print("Variable content does not match. There may be an issue with saving/loading.")

Variable content matches. The file was saved and loaded successfully.


Make a function `save(leaders_per_country)` to call this code easily.

In [50]:
# 3 lines
import json

def save(leaders_per_country: dict) -> None:
    """
    Function to save a dictionary of state leader data to a json file, reloads the file to check the content and prints out a message, 
    informing, wether or not the saving process was successful.

    Args:
        leaders_per_country (dict): Dictionary containing data of state leaders per country.

    Return:
        None
    """

    # Save dictionary to json file
    with open ("leaders.json", "w") as json_file:
        json.dump(leaders_per_country, json_file, indent=4)
    
    # Read the data back from leaders.json
    with open("leaders.json", "r") as json_file:
        loaded_data = json.load(json_file)
    
    # Check, if the content loaded is the same as the original dictionary
    if loaded_data == leaders_per_country:
        print("Variable content matches. The file was saved and loaded successfully.")
    else:
        print("Variable content does not match. There may be an issue with saving/loading.")
    

In [51]:
# Call the function (1 line)
save(leaders_per_country)

Variable content matches. The file was saved and loaded successfully.


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!